In [22]:
from glob import glob

from collections import defaultdict
import json
import os

In [23]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://husein-ai6-aiservices916116520.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''
engine = 'gpt-4'

In [24]:
questions = defaultdict(set)
q = []
for f in glob('base-questions-menengah-*/*'):
    b = os.path.split(f)[0]
    with open(f) as fopen:
        data = json.load(fopen)
    data = '\n'.join(data)
    data = data.split('\n\n')
    if len(data) == 1:
        data = data[0].split('\n')
    data = ['.'.join(d.split('.')[1:]).strip() for d in data]
    data = [d for d in data if len(d) > 100]
    data = set(data)
    questions[b] |= data

In [25]:
questions.keys()

dict_keys(['base-questions-menengah-kimia-tingkatan-5-malaysia', 'base-questions-menengah-prinsip-akaun-tingkatan-5-malaysia', 'base-questions-menengah-pengajian-keusahawanan-tingkatan-4-malaysia', 'base-questions-menengah-biologi-tingkatan-5-malaysia', 'base-questions-menengah-sains-tingkatan-2-malaysia', 'base-questions-menengah-sejarah-tingkatan-2-malaysia', 'base-questions-menengah-sejarah-tingkatan-3-malaysia', 'base-questions-menengah-matematik-tingkatan-5-malaysia', 'base-questions-menengah-matematik-tingkatan-1-malaysia', 'base-questions-menengah-matematik-tingkatan-3-malaysia', 'base-questions-menengah-sejarah-tingkatan-1-malaysia', 'base-questions-menengah-sains-tingkatan-1-malaysia', 'base-questions-menengah-sejarah-tingkatan-4-malaysia', 'base-questions-menengah-fizik-tingkatan-4-malaysia', 'base-questions-menengah-matematik-tingkatan-2-malaysia', 'base-questions-menengah-sejarah-tingkatan-5-malaysia', 'base-questions-menengah-fizik-tingkatan-5-malaysia', 'base-questions-me

In [26]:
total_questions = 0
all_questions = []
for k, v in questions.items():
    total_questions += len(v)
    all_questions.extend([(k, v_) for v_ in v])
    
total_questions

8406

In [27]:
all_questions = sorted(all_questions)

In [7]:
!mkdir answer-question-chatgpt4-menengah-part2
# !rm -rf answer-question-chatgpt4-menengah-part2/*

mkdir: cannot create directory ‘answer-question-chatgpt4-menengah-part2’: File exists


In [8]:
from tqdm import tqdm

for i in tqdm(range(len(all_questions))):
    filename = f'answer-question-chatgpt4-menengah-part2/{i}.json'
    if os.path.exists(filename):
        continue
        
    try:
        message_text = [
            {"role":"user","content": f"{all_questions[i][1]}, jawab dalam bahasa melayu"},
        ]
        completion = openai.ChatCompletion.create(
          engine="gpt-4",
          messages = message_text,
          temperature=1.0,
          max_tokens=1024,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        splitted = completion.choices[0]['message']['content'].split('\n')
    except:
        splitted = None
    with open(filename, 'w') as fopen:
        json.dump(splitted, fopen)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 8406/8406 [31:40:40<00:00, 13.57s/it]


In [28]:
qa, texts = [], []
for i in range(len(all_questions)):
    q = all_questions[i][1]
    
    filename = f'answer-question-chatgpt4-menengah-part2/{i}.json'
    if not os.path.exists(filename):
        continue
    with open(filename) as fopen:
        answer = json.load(fopen)
    
    if answer is None:
        continue
        
    answer = '\n'.join(answer)
    
    qa.append({
        'question': q.strip(),
        'answer': answer.strip()
    })
    
    texts.extend([q.strip(), answer.strip()])
    
len(qa)

8402

In [29]:
print(qa[0]['answer'])

Perubahan musim memainkan peranan penting dalam kitar hidup nyamuk Aedes aegypti, yang merupakan vektor utama penyakit demam denggi di Malaysia. Berikut adalah analisis mengenai bagaimana perubahan musim dapat mempengaruhi kitar hidup nyamuk ini:

**Musim Hujan:**

1. *Pembiakan*: Nyamuk Aedes aegypti membiak di air yang tergenang. Musim hujan menyediakan banyak tempat pembiakan yang sesuai seperti genangan air, baldi, pasu bunga, dan bekas yang terkumpul air hujan, membolehkan populasi nyamuk meningkat dengan cepat.

2. *Tempoh Kitar Hidup*: Suhu yang hangat dan kelembapan tinggi semasa musim hujan menyokong perkembangan larva menjadi nyamuk dewasa dengan lebih pantas.

3. *Kemunculan Jentik-jentik*: Air yang terkumpul setelah hujan menyediakan habitat yang ideal untuk jentik-jentik berkembang, mempercepat proses pematangan dari larva ke nyamuk dewasa.

4. *Penyebaran Penyakit*: Kenaikan jumlah nyamuk dewasa bermakna peningkatan risiko penyebaran demam denggi jika nyamuk tersebut meng

In [30]:
with open('answer-question-chatgpt4-menengah-part2.texts', 'w') as fopen:
    for t in set(texts):
        fopen.write(f'{json.dumps(t)}\n')

In [31]:
len(set(texts))

16803

In [21]:
!cp answer-question-chatgpt4-menengah-part2.texts ../ctranslate2

In [32]:
mapping = {}
with open('../ctranslate2/answer-question-chatgpt4-menengah-part2.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']
len(mapping)

16803

In [33]:
with open('malaysian-general-qa-sekolah-menengah.jsonl', 'w') as fopen:
    for q in qa:
        q['question_ms'] = mapping.get(q['question'])
        q['answer_ms'] = mapping.get(q['answer'])
        fopen.write(f'{json.dumps(q)}\n')

In [34]:
q

{'question': 'Terangkan tentang pendudukan Jepun di Tanah Melayu dan bagaimana ia memberi kesan kepada masyarakat setempat.',
 'answer': 'Pendudukan Jepun di Tanah Melayu bermula pada 8 Disember 1941 ketika Jepun menyerang Pantai Timur Tanah Besar Semenanjung Malaya semasa Perang Dunia Kedua. Kejatuhan Singapura pada 15 Februari 1942 menandakan kejayaan Jepun menguasai seluruh Tanah Melayu. Pendudukan ini berakhir pada 12 September 1945 dengan penyerahan Jepun kepada tentera Berikat.\n\nSelama pendudukan yang berlangsung selama tiga setengah tahun, penduduk Tanah Melayu mengalami perubahan besar dalam kehidupan sosial, ekonomi, dan politik mereka. Berikut adalah beberapa kesan pendudukan Jepun terhadap masyarakat setempat:\n\n1. Kesulitan Ekonomi dan Makanan: Pendudukan Jepun menyebabkan kekurangan makanan dan barangan penting lain. Jepun mengutamakan sumber dan hasil bumi Tanah Melayu untuk keperluan perang mereka, menyebabkan penduduk setempat mengalami kesukaran mendapatkan bahan ma

In [35]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='malaysian-general-qa-sekolah-menengah.jsonl',
    path_in_repo='malaysian-general-qa-sekolah-menengah.jsonl',
    repo_id='mesolitica/chatgpt4-malaysian-general-qa',
    repo_type='dataset',
)

malaysian-general-qa-sekolah-menengah.jsonl:   0%|          | 0.00/35.6M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/chatgpt4-malaysian-general-qa/blob/main/malaysian-general-qa-sekolah-menengah.jsonl'